# Introduction to Vcc Glitch Attacks

**This tutorial assumes the following hardware setup:**
* CW308_STM32F0 target
* Short SMA cable between CWLite and CW308
* 7.37MHz Xtal in X1 on CW308
* CW308 J3 set to Crystal

**This tutorial may be possible with other hardware setups, but workarounds may be needed**

## Background on Vcc (Power) Glitching

The previous clock glitching tutorials looked into the assumption of a constant clock. But instead we can modify the voltage of the device, causing for example a failure to correctly read a memory location or otherwise cause havoc with the proper functioning.

An example of a successful Vcc glitch is shown in the following figures (Vcc in blue, clock in red):

![](https://wiki.newae.com/images/4/4f/Vccglitch_working.png)

![](https://wiki.newae.com/images/6/60/Vccglitch_working_zoom.png)

Like with clock glitching, Vcc glitching is highly sensitive to glitch offset and width. While the above glitch was successful, the following was not:

![](https://wiki.newae.com/images/b/b6/Vccglitch_notworking_zoom.png)

Vcc glitching is also very sensitive to the shape of the glitch: things like board layout and the distance between where the glitch is inserted and the target can make the difference between successful and unsuccessful glitches.

Despite these additional complications, Vcc glitching is an extremely useful tool as it allows attacks on targets that do not run off of external clock inputs.

### Background on Glitch Generation

For more details, please see [Fault_1-Introduction_to_Clock_Glitching](Fault_1-Introduction_to_Clock_Glitch_Attacks.ipynb), this tutorials assumes you have already performed the clock glitching tutorial.

The glitch generation hardware is the same as used in the clock glitching attack. The generated glitches are synchronous to the device clock, and inserted at a precise offset from the clock edge.

Glitches can be inserted continuously or triggered by some event. The following figure shows the generation of two glitches:

![](https://wiki.newae.com/images/9/95/Glitchgen-mux-glitchonly.png)

The VCC glitching method here uses an electronic switch (a MOSFET) to short the power line to GND at specific instances. The following figure shows the basic function of this system:

![](https://wiki.newae.com/images/8/82/Glitch-vccglitcher.png)

This method allows use with the standard side-channel analysis development board, which has resistors inserted into the VCC lines already. The downside of this method is that it can only generate short glitches, since the power consumption through the shunt resistor will short out the resistor.

The MOSFET glitching hardware is built into the ChipWhisperer-Lite (both CW1173 and CW1180) board. The ChipWhisperer-Capture Rev2 uses an external VCC glitching board.

## Setting up Firmware

During this tutorial, we will once again be working off the `glitch-simple` project. We'll start by creating a new firmware project:

In [1]:
%%bash
cd ../../hardware/victims/firmware/
mkdir -p glitch-simple-lab2 && cp -r glitch-simple/* $_

For this tutorial we'll be using the STM32F0 target, since that board often works well for Vcc glitching.

In [2]:
PLATFORM = "CWLITEARM"
CRYPTO_TARGET="NONE"

Now navigate to the `glitch-simple-lab2` folder and open `glitchsimple.c` in a code editor. Find the `glitch_infinite()` function:

```C
void glitch_infinite(void)
{
    char str[64];
    unsigned int k = 0;
    //Declared volatile to avoid optimizing away loop.
    //This also adds lots of SRAM access
    volatile uint16_t i, j;
    volatile uint32_t cnt;
    while(1){
        cnt = 0;
        trigger_high();
        trigger_low();
        for(i=0; i<200; i++){
            for(j=0; j<200; j++){
                cnt++;
            }
        }
        sprintf(str, "%lu %d %d %d\n", cnt, i, j, k++);
        uart_puts(str);
    }
}
```

As you can see, this function enters into an infinite loop with two inner loops that increment three variables (`cnt`, `i`, and `j`).  These are sent back over serial along with an overall loop counter. During normal operation, we should receive `40000 200 200 $k` (where `$k` is the value of the loop counter `k`). Our objective will be to insert a Vcc glitch such that one or more of the numbers that we get back are incorrect.

Navigate to `main()` and change the `glitch1()` call to instead call `glitch_infinite()`. The below block will also do this:

In [3]:
%%bash
# replaces glitch1() with glitch_infinite()
cd ../../hardware/victims/firmware/glitch-simple-lab2
awk '{gsub(/glitch1\(\)/, "glitch_infinite\(\)")}1' glitchsimple.c > tmp.c 2>/dev/null
rm glitchsimple.c
mv tmp.c glitchsimple.c

Then build the firmware:

In [4]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../hardware/victims/firmware/glitch-simple-lab2
make PLATFORM=$1 CRYPTO_TARGET=$2 MCU_CLK=INT

rm -f -- glitchsimple-CWLITEARM.hex
rm -f -- glitchsimple-CWLITEARM.eep
rm -f -- glitchsimple-CWLITEARM.cof
rm -f -- glitchsimple-CWLITEARM.elf
rm -f -- glitchsimple-CWLITEARM.map
rm -f -- glitchsimple-CWLITEARM.sym
rm -f -- glitchsimple-CWLITEARM.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- glitchsimple.s simpleserial.s stm32f3_hal.s stm32f3_hal_lowlevel.s stm32f3_sysmem.s
rm -f -- glitchsimple.d simpleserial.d stm32f3_hal.d stm32f3_hal_lowlevel.d stm32f3_sysmem.d
rm -f -- glitchsimple.i simpleserial.i stm32f3_hal.i stm32f3_hal_lowlevel.i stm32f3_sysmem.i
.
-------- begin --------
arm-none-eabi-gcc (GNU Tools for Arm Embedded Processors 7-2018-q2-update) 7.3.1 20180622 (release) [ARM/embedded-7-branch revision 261907]
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling C: glitchsimple.c
arm-none-eabi-gcc -c -mcpu=co

## Attack Script

### Setup

Now that we've studied the code and have an objective, we can start building our attack script. We'll start by connecting to and setting up the ChipWhisperer, then programming it. As usual, make sure you modify `fw_path` with the path to the file you built in the last step.

In [2]:
import chipwhisperer as cw
scope = cw.scope()
target = cw.target(scope)

In [3]:
%run "Helper_Scripts/Setup_Target_Generic.ipynb"

In [4]:
fw_path = "../../hardware/victims/firmware/glitch-simple-lab2/glitchsimple-CWLITEARM.hex"
prog = cw.programmers.STM32FProgrammer

In [5]:
cw.programTarget(scope, prog, fw_path)

Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to programming 6599 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 6599 bytes


Like with clock glitching, Vcc glitching may crash the target, requiring a reset:

In [6]:
import time
def reset_target(scope):
    scope.io.nrst = 'low'
    #scope.io.pdic = 'low'
    time.sleep(0.05)
    scope.io.nrst = 'high'
    #scope.io.pdic = 'high'

Now that we have some of the basic setup done, let's make sure the firmware works as we expect. If we reset the target and wait a second, then print the serial data we got back, we should see a number of lines of the form `40000 200 200 $k`.

In [7]:
reset_target(scope)
target.ser.flush()
time.sleep(1)
num_char = target.ser.inWaiting()
resp = target.ser.read(num_char)
print(resp)

40000 200 200 0
40000 200 200 1
40000 200 200 2
40000 200 200 3
40000 200 200 4
40000 200 200 5
40000 200 200 6
40000 200 200 7


### Glitch Setup

Since Vcc glitching requires greater precision than clock glitching, we'll need to take a different approach than we did with the last tutorial. 

Our first major change is to the clock of the ChipWhisperer. Instead of running at 7.37MHz, we'll instead run at a much higher clock rate (257.95MHz or 35\*7.37MHz). This will give us much better control over the width, shape, and placement of our glitches. Real targets often have their own independent clock setup anyway, so this isn't an obstacle to a real attack.

Note that this clock frequency is higher than the ADC on the ChipWhisperer-Lite can run off of, so you'll likely see the red ADC DCM Lock Failed LED on your ChipWhisperer light up.

In [16]:
scope.clock.clkgen_freq = 256000000

Next, we'll setup the glitch module itself. Most of these settings should look familiar from the previous tutorial with a few new additions:

* Instead of setting the clock source for the target to be the glitch module, we instead set the low power MOSFET's input to be the glitch module by setting `scope.io.glitch_lp` to `True`. The ChipWhisperer-Lite also has a high power MOSFET, but we won't be using that in this tutorial.
* Instead of setting the glitch output to something like "clock_xor", we instead set it to "glitch_only", since we don't want Vcc of the target to be oscillating with our clock.

For the more specific settings (offset, width, repeat, etc), this will depend on both the target and when you got your CW-Lite: Newer versions of the CW-Lite use a different glitch MOSFET, which changes the settings required for getting a glitch.

If you're unsure which version you have, try running the old settings. These settings always cause the target to crash if used with a new CW-Lite.

#### Old CW-Lite Settings

In [ ]:
scope.glitch.clk_src = "clkgen"
scope.glitch.output = "glitch_only"
scope.glitch.trigger_src = "ext_single"
scope.glitch.width = 28.5
scope.glitch.offset = 8.2
scope.glitch.repeat = 14
scope.io.glitch_lp = True
print(scope.glitch)

#### New CW-Lite Settings

In [9]:
scope.glitch.clk_src = "clkgen"
scope.glitch.output = "glitch_only"
scope.glitch.trigger_src = "ext_single"
scope.glitch.width = 45.3
scope.glitch.offset = 14
scope.glitch.repeat = 9
scope.io.glitch_lp = True
print(scope.glitch)

clk_src     = clkgen
width       = 45.3125
width_fine  = 0
offset      = 14.0625
offset_fine = 0
trigger_src = ext_single
arm_timing  = after_scope
ext_offset  = 0
repeat      = 9
output      = glitch_only



### Glitching a Single Point

Unlike with the previous tutorial, we don't control when the device sends serial data back to us. This means we'll need to parse the data we get back. To make things a little simpler, we'll start off by attacking a single point multiple times.

This point will be target specific, but for the STM32F0, this point works well:

In [18]:
#scope.glitch.ext_offset = 2181 #old CW-Lite
scope.glitch.ext_offset = 2028 #new CW-Lite

We start our attack off by flushing the ChipWhipserer's serial buffer:

```Python
target.ser.flush()
```

Next, we'll set our trigger source to be "ext_continuous". This differs from "ext_single" in that the ChipWhisperer doesn't need to be armed to insert a glitch, making our loop a little simpler:

```Python
scope.glitch.trigger_src = "ext_continuous"
```

A key part of parsing the serial data is to be able to read a line of data (data terminated with "\n"). We can do that by reading back data until we get a newline character ("\n"):

```Python
while "\n" not in line:
    time.sleep(0.1)
    num_char = target.ser.inWaiting()
    line += target.ser.read(num_char)
```

This needs to be repeated twice in our loop: once at the start to make sure we're on a newline (so we don't look at the wrong numbers for our glitch) and again to actually read the line. For the first read, we also need to make sure we keep any characters after the newline, as this will be the start of the actual line we parse. All together, this looks like:

```Python
line = ""
while "\n" not in line:
    time.sleep(0.1)
    num_char = target.ser.inWaiting()
    line += target.ser.read(num_char)
lines = line.split("\n") 
if len(lines) > 1:
    line = lines[-1]
else:
    line = ""

while "\n" not in line:
    time.sleep(0.1)
    num_char = target.ser.inWaiting()
    line += target.ser.read(num_char)
```

Now that we have our line of data we can parse it by splitting it up via spaces to get each number. 

After the loop ends, we'll need to set our trigger back to "ext_single" to stop the glitches from continuing.

All together (with some additional error checking), this looks like:

In [19]:
from tqdm import tnrange
reset_target(scope)
target.ser.flush()
scope.glitch.trigger_src = "ext_single"
scope.glitch.trigger_src = "ext_continuous"

for j in tnrange(20):
    line = ""
    while "\n" not in line:
        time.sleep(0.1)
        num_char = target.ser.inWaiting()
        line += target.ser.read(num_char)
    lines = line.split("\n") 
    if len(lines) > 1:
        line = lines[-1]
    else:
        line = ""
    
    while "\n" not in line:
        time.sleep(0.1)
        num_char = target.ser.inWaiting()
        line += target.ser.read(num_char)
        
    if "hello" in line:
        print("Target crashed")
    nums = line.split(" ")
    try:
        if int(nums[0]) != 40000:
            print(line)
    except ValueError as e:
        continue
    
scope.glitch.trigger_src = "ext_single"

You should hopefully see some glitched lines printed after running the above block. If not, don't worry -- we'll be scanning ext_offset next, which should help find glitch locations if they moved.

### Scanning Ext Offsets

Adding the ability to scan ext offsets is rather easy: we just need to wrap our existing code in another loop:

In [20]:
from tqdm import tnrange
target.ser.flush()
scope.glitch.repeat = 10
for ext in tnrange(2025, 2030, desc="Ext offset"):
    scope.glitch.ext_offset = ext
    target.ser.flush()
    scope.glitch.trigger_src = "ext_continuous"

    for j in tnrange(10, leave=False, desc="Glitch attempt"):
        line = ""
        while "\n" not in line:
            time.sleep(0.1)
            num_char = target.ser.inWaiting()
            if num_char == 0:
                scope.io.glitch_lp = False
                time.sleep(0.01)
                scope.io.glitch_lp = True
                break
            line += target.ser.read(num_char)
        lines = line.split("\n") 
        if len(lines) > 1:
            line = lines[-1]
        else:
            line = ""

        while "\n" not in line:
            time.sleep(0.1)
            num_char = target.ser.inWaiting()
            if num_char == 0:
                scope.io.glitch_lp = False
                time.sleep(0.01)
                scope.io.glitch_lp = True
                break
            line += target.ser.read(num_char)

        nums = line.split(" ")
        if "hello" in line:
            print("Target crashed @ {}".format(ext))
        try:
            if int(nums[0]) != 40000:
                print("Glitched @ {}".format(ext))
                print("Text: " + line)
        except ValueError as e:
            continue

    scope.glitch.trigger_src = "ext_single"

If you're still unable to glitch, there's a few things you can try:
* Increasing the range of ext offsets that we scan
* Increasing the number of attempts at each ext offset

It's also possible that, though you have the right glitch location, your glitch settings don't produce glitches very often.

### Improving Glitch Settings

One more thing we can do to improve our glitch success rate is to modify our glitch settings (width and offset) while holding the ext offset constant. 

Similar to [Fault_1](Fault_1-Introduction_to_Clock_Glitch_Attacks.ipynb), let's also measure our success rate with each glitch setting, as well as our crash rate, then print them at the end.

If you found an ext offset that worked well for you, be sure to fill it in below:

In [64]:
scope.glitch.trigger_src = "ext_single"

In [32]:
from tqdm import tnrange, tqdm_notebook
from collections import namedtuple

scope.glitch.ext_offset = 2028

Range = namedtuple('Range', ['min', 'max', 'step'])
width_range = Range(43.5, 46, 0.4)
offset_range = Range(-40, 40, 2)
N = 10
scope.glitch.repeat = 9
scope.glitch.trigger_src = "ext_single"
reset_target(scope)
target.ser.flush()

scope.glitch.trigger_src = "ext_continuous"

glitches = []
scope.glitch.offset = offset_range.min

t_offset = tqdm_notebook(total=int((offset_range.max-offset_range.min)/offset_range.step) + 1, desc="Offset")

while scope.glitch.offset < offset_range.max:
    scope.glitch.width = width_range.min
    t_width = tqdm_notebook(total=int((width_range.max-width_range.min)/width_range.step), leave=False, desc="Width")
    while scope.glitch.width < width_range.max:
        successes = 0
        crashes = 0
        for j in tnrange(N, leave=False, desc="Attempt"):
            line = ""
            while "\n" not in line:
                time.sleep(0.1)
                num_char = target.ser.inWaiting()
                if num_char == 0:
                    scope.io.glitch_lp = False
                    time.sleep(0.01)
                    scope.io.glitch_lp = True
                    break
                line += target.ser.read(num_char)
            lines = line.split("\n") 
            if len(lines) > 1:
                line = lines[-1]
            else:
                line = ""

            while "\n" not in line:
                time.sleep(0.1)
                num_char = target.ser.inWaiting()
                if num_char == 0:
                    scope.io.glitch_lp = False
                    time.sleep(0.01)
                    scope.io.glitch_lp = True
                    break
                line += target.ser.read(num_char)

            nums = line.split(" ")
            if "hello" in line:
                crashes += 1
                print("Target crashed")
            try:
                if nums[0] == "":
                    continue
                if int(nums[0]) != 40000:
                    print(f"Success @ {scope.glitch.width}, {scope.glitch.offset}")
                    successes += 1
            except ValueError as e:
                continue
        glitches.append([scope.glitch.width, scope.glitch.offset, successes / N, crashes / N])
        scope.glitch.width += width_range.step
        t_width.update()
        
    scope.glitch.offset += offset_range.step
    t_offset.update()
    t_width.close()

t_width.close()
t_offset.close()
scope.glitch.trigger_src = "ext_single"

Target crashed
Target crashed


Target crashed


Target crashed


Target crashed
Target crashed


Target crashed
Target crashed


Target crashed


Target crashed
Target crashed
Target crashed


Target crashed


Traceback (most recent call last):
  File "c:\users\user\code\term2\chipwhisperer\software\chipwhisperer\hardware\naeusb\naeusb.py", line 277, in txrx
    response = self.usbdev().ctrl_transfer(payload[0], payload[1], payload[2], payload[3], payload[4], timeout=self._timeout)
  File "c:\users\user\appdata\local\programs\python\python37-32\lib\site-packages\pyusb-1.0.2-py3.7.egg\usb\core.py", line 1043, in ctrl_transfer
    self.__get_timeout(timeout))
  File "c:\users\user\appdata\local\programs\python\python37-32\lib\site-packages\pyusb-1.0.2-py3.7.egg\usb\backend\libusb0.py", line 593, in ctrl_transfer
    timeout
  File "c:\users\user\appdata\local\programs\python\python37-32\lib\site-packages\pyusb-1.0.2-py3.7.egg\usb\backend\libusb0.py", line 431, in _check
    raise USBError(errmsg, ret)
usb.core.USBError: [Errno None] b'usb_control_msg: sending control message failed, win error: The device is not connected.\r\n'


USBError: [Errno None] None

Then, sorting by success rate:

In [28]:
def sort_glitch(glitch):
    return glitch[3]

glitches.sort(key=sort_glitch,reverse=True)
for glitch in glitches:
    print(glitch)

[39.84375, 1.171875, 0.0, 1.0]
[39.84375, 3.125, 0.0, 1.0]
[39.84375, 5.078125, 0.0, 1.0]
[40.234375, 5.078125, 0.0, 1.0]
[39.84375, 8.984375, 0.0, 1.0]
[40.234375, 8.984375, 0.0, 1.0]
[39.84375, 10.9375, 0.0, 1.0]
[39.84375, 12.890625, 0.0, 1.0]
[40.234375, 12.890625, 0.0, 1.0]
[39.84375, 14.84375, 0.0, 1.0]
[39.84375, 16.796875, 0.0, 1.0]
[39.84375, 18.75, 0.0, 1.0]
[40.234375, 18.75, 0.0, 1.0]
[39.84375, 20.703125, 0.0, 1.0]
[40.234375, 20.703125, 0.0, 1.0]
[40.625, 20.703125, 0.0, 1.0]
[41.015625, 20.703125, 0.0, 1.0]
[41.40625, 20.703125, 0.0, 1.0]
[41.796875, 20.703125, 0.0, 1.0]
[39.84375, 22.65625, 0.0, 1.0]
[40.234375, 22.65625, 0.0, 1.0]
[40.625, 22.65625, 0.0, 1.0]
[39.84375, 24.609375, 0.0, 1.0]
[40.234375, 24.609375, 0.0, 1.0]
[39.84375, 26.5625, 0.0, 1.0]
[39.84375, 28.515625, 0.0, 1.0]
[39.84375, 30.46875, 0.0, 1.0]
[39.84375, 32.421875, 0.0, 1.0]
[40.234375, 32.421875, 0.0, 1.0]
[39.84375, 34.375, 0.0, 1.0]
[40.234375, 36.328125, 0.0, 1.0]
[39.84375, 38.28125, 0.0, 1.0]

[45.703125, 24.609375, 0.0, 0.0]
[42.578125, 26.5625, 0.0, 0.0]
[43.359375, 26.5625, 0.0, 0.0]
[43.75, 26.5625, 0.0, 0.0]
[44.140625, 26.5625, 0.0, 0.0]
[44.53125, 26.5625, 0.0, 0.0]
[44.921875, 26.5625, 0.0, 0.0]
[45.3125, 26.5625, 0.0, 0.0]
[45.703125, 26.5625, 0.0, 0.0]
[43.359375, 28.515625, 0.0, 0.0]
[43.75, 28.515625, 0.0, 0.0]
[44.140625, 28.515625, 0.0, 0.0]
[44.53125, 28.515625, 0.0, 0.0]
[44.921875, 28.515625, 0.0, 0.0]
[45.3125, 28.515625, 0.0, 0.0]
[45.703125, 28.515625, 0.0, 0.0]
[42.578125, 30.46875, 0.0, 0.0]
[43.359375, 30.46875, 0.0, 0.0]
[43.75, 30.46875, 0.0, 0.0]
[44.140625, 30.46875, 0.0, 0.0]
[44.53125, 30.46875, 0.0, 0.0]
[44.921875, 30.46875, 0.0, 0.0]
[45.3125, 30.46875, 0.0, 0.0]
[45.703125, 30.46875, 0.0, 0.0]
[43.359375, 32.421875, 0.0, 0.0]
[43.75, 32.421875, 0.0, 0.0]
[44.140625, 32.421875, 0.0, 0.0]
[44.53125, 32.421875, 0.0, 0.0]
[44.921875, 32.421875, 0.0, 0.0]
[45.3125, 32.421875, 0.0, 0.0]
[45.703125, 32.421875, 0.0, 0.0]
[42.96875, 34.375, 0.0, 0.0]


### Going Further

There's a lot more you can do with this attack:
* If you still weren't able to get any glitches, create an attack loop that scans ext offset, offset, width, and possibly even repeat
* Glitching different instructions will produce different results. Try using your best glitch settings from the previous part and scanning much larger ext offset ranges to see if you can find different glitch effects
    * During our ext offset loop, we only checked for glitches in the first number (40000). You may want to examine the other numbers for glitches as well
    * The ChipWhisperer is running at 35 times the clock speed of the target. If you want to cover a lot of ground very quickly, you can start on a successful location and advance ext offset by 35 each cycle instead of 1. Note that not all glitch locations will be 35 away from the last, so you will miss glitches this way
* Open the listing file (`.lss`) and view the assembly of the `glitch_infinite()` function. Can you explain the different glitch effects you saw?
    
If you don't want to go searching, here's a few good ranges you can try here that may produce different results (on the STM32F0) with the old CW-Lite:
* [2036, 2038]
* [2180, 2182]
* [2317, 2319]

and with the new one:
* [2185, 2187]
* [2361, 2363]
* [2535, 2536]
* [2675, 2677]

Otherwise, we're done with the tutorial. You can now disconnect from the ChipWhisperer:

In [ ]:
scope.dis()
target.dis()

## Conclusion

With the tutorial now finished, you should have some Vcc glitching experience under your belt. If you're interested in doing more Vcc glitching, you may want to try [Tutorial A9](https://wiki.newae.com/Tutorial_A9_Bypassing_LPC1114_Read_Protect) from the ChipWhisperer Wiki, which uses Vcc glitching to bypass code readout protection on an LPC1114 (requires an LPC1114 dev board). You may also want to try glitching some of the other functions in `glitchsimple.c`. If you have a Raspberry Pi, you can also attempt the attack described [here](https://wiki.newae.com/Tutorial_A3_VCC_Glitch_Attacks#Glitching_More_Advanced_Targets:_Raspberry_Pi) (though you'll need to transfer the steps from the old GUI over to Jupyter).